In [1]:
import os, re, sys
from datetime import date
import importlib
import jpnotebooks.Software.SDLC_traceability_tools.issuetracker_item_extracter2 as iie
import jpnotebooks.Software.SDLC_traceability_tools.user_tests_parser as utp
import scripts.tools.redmine_cache as rmc
from scripts.software_frs import frs_traceability2 as FRS

iie = importlib.reload(iie)
utp = importlib.reload(utp)
rmc = importlib.reload(rmc)

UserTestsParser = utp.UserTestsParser
RequirementExtracter = iie.RequirementExtracter

In [2]:
_cache = None
_age = None
_client = None

def _get_client():
    global _client
    if _client is None:
        _client = rmc.RedmineClient()
    return _client

def _download_issues():
    _client = _get_client()
    return _client.get_filtered([('project.identifier', '==', 'pbssoftware')])

def load_issues():
    return _download_issues()


In [3]:
import re
class WebFRSIssuetrackerParser(iie.IssuetrackerParser):
    def __init__(self, types=("URS", "FRS", "SDS", "SWDS")):
        types = list(types) + ["3.0WebFRS"]
        super().__init__(types)
        
    def _get_result_for_line(self, line):
        """ identical to parent function, but checks the type after
        scanning the line to return only 3.0WebFRS items, converted
        to plain FRS items. 
        """
        if not line or line.isspace():
            return self._EMPTY_LINE, None, None, None, True

        m = self._item_match(line)
        if m is None: 
            return self._RAW_LINE, "", "", line.strip(), False
        
        dash1, typ, num, text, dash2 = m.groups()
        if typ != '3.0WebFRS':
            return self._EMPTY_LINE, None, None, None, True
        else:
            typ = 'FRS'
            num = "3." + num
        cancel = dash1 == dash2 and dash1 != ""
        return self._REQ_RESULT, typ, num, text, cancel

In [110]:
rtags = [
    "URS",
    "FRS",
    "SWDS",
    "CS",
    "BUG"
]

ttags = [
    "USR"
]

_ignore = {3194, 3287}
_sprints = {
    'Legacy',
    '3.0',
    '3.0.1',
    '3.0.2',
    '3.0.3',
    '3.0.4',
    '3.0.5',
    '3.0.6',
    '3.0.7'
}

def relevant(i):
    return i.sprint_milestone.name in _sprints and i.id not in _ignore and i.status != "Rejected"

def relevent38(i):
    return i.sprint_milestone.name == "3.0.8" and i.id not in _ignore and i.status != "Rejected"

In [126]:
iie = importlib.reload(iie)
utp = importlib.reload(utp)
UserTestsParser = utp.UserTestsParser
RequirementExtracter = iie.RequirementExtracter

force = True
force = False
issues = load_issues()
issues2 = list(filter(relevant, issues.values()))

parser = iie.IssuetrackerParser(rtags)
reqs = parser.parse_all(issues2)

parser2 = UserTestsParser()
tests = parser2.parse_excel("C:\\Users\\Nathan\\Documents\\Dropbox\\FRS\\FRS Tests 190226.xlsx")

parser3 = WebFRSIssuetrackerParser(rtags)
reqs2 = parser3.parse_all(issues2)

allitems = reqs + tests + reqs2

def missing_parents(req, ref, reqs):
    if req.type == "TEST":
        return 'ignore'
    else:
        return 'fix'

rex = RequirementExtracter(rtags, ttags)
rex.config.missing_parents = missing_parents
rows = rex.extract(allitems)

In [127]:
from officelib.xllib import *
class JamesExcelFileRequirementsParser():
    def __init__(self, rtags):
        self.rtags = rtags
    
    def extract(self, fp, sprint="3.0.8", close=False):
        xl = Excel()
        with screen_lock(xl):
            wb = xl.Workbooks.Open(fp)
            ws = wb.Worksheets("specs list")
            ws2 = wb.Worksheets("issues")
            issues = self._parse_sprints(ws2, sprint)
            reqs = self._extract(ws, issues)
            if close:
                wb.Close(False)
        return reqs
    
    def _parse_sprints(self, ws, sprint):
        cells = ws.Cells
        cr = cells.Range
        c1 = cr("A2")
        c2 = c1.End(xlc.xlDown).End(xlc.xlToRight)
        allsprints = cr(c1, c2).Value2
        irel = set()
        for iss, isprint, text, priority in allsprints:
            if isprint == sprint:
                irel.add(iss)
        return irel
    
    def _extract(self, ws, issues):
        cells = ws.Cells
        cr = cells.Range
        c1 = cr("A1")
        er = c1.End(xlc.xlDown).Row
        c2 = c1.End(xlc.xlToRight).GetOffset(er - c1.Row, 0)
        data = cr(c1.GetOffset(1, 0), c2).Value2
        return self._get_data(data, issues)
    
    def _get_data(self, data, issues):
        out = []
        for iss, num, text, refs, uprefs, iss2 in data:
            if iss not in issues:
                continue
            if num.startswith("<"):
                continue
            refs = [s.strip() for s in (refs or "").replace(",", "").split()]
            if num.startswith("FRS") or num.startswith("URS"):
                typ = num[:3]
                num = num[3:]
            else:
                typ = "FRS"
            ref = iie.Reference(typ, num, False, refs)
            out.append(ref)
        return out

In [338]:
issues = load_issues()
issues3 = list(filter(relevent38, issues.values()))
newitems = parser.parse_all(issues3)
newitems2 = JamesExcelFileRequirementsParser(rtags).extract("C:\\Users\\Nathan\\Documents\\Personal\\test\\NewPumpFeatures.xlsx")
newitems.extend(newitems2)

In [263]:
import networkx as nx

import itertools

def impacted_nodes(G, node):
    a = nx.shortest_path_length(G, target=node)
    d = nx.shortest_path_length(G, source=node)
    return {n for n in itertools.chain(a, d)}

def impact_graph(G, node):
    r = impacted_nodes(G, node)
    return G.subgraph(r)

def items_to_graph(items, reqs):
    G = nx.DiGraph()
    for req in items:
        if not req.obs:
            for ref in req.refs:
                if ref in reqs:
                    G.add_edge(ref, req.tag)
    return G

def dictify(items):
    out = {}
    for i in items:
        if not i.obs:
            if i.tag in out:
                print("Warning: duplicate for tag: %s"%i.tag)
            out[i.tag] = i
    return out

def add_implicit_local_refs(items):
    for req in items:
        hpref = req.tag.rsplit(".", 1)[0]
        if hpref != req.tag:
            req.refs.add(hpref)
            
def make_impact_graph(G, node, file):
    SG = impact_graph(G, node)
    nx.write_graphml(G, file)

In [141]:
FRS = importlib.reload(FRS)
_imkey = re.compile(r"(%s)(\d+)\.?([\d\.]*)" % "|".join(['URS', 'FRS', 'SDS'])).match

def key_match(key):
    m = _imkey(key)
    if m:
        type, first, others = m.groups()
        return type+first, others
    return key, ""

def root_for_type(items, typ):
    root = FRS.Root(key_match)
    for req in items:
        if req.type == typ and not req.obs:
            root.add(req.tag, 0)
    return root

In [84]:
add_implicit_local_refs(allitems)
add_implicit_local_refs(newitems)
urs = root_for_type(allitems, "URS")
G = items_to_graph(allitems)
reqs = dictify(allitems)

with open("test.nnf", 'w') as f:
    for u in urs.iter():
        if u.is_leaf():
            desc = nx.shortest_path_length(G, source=u.id)
            G2 = condensed_graph(desc, reqs)
            f.write("topN SWDev xx Req:%s\n"%(u.id))
            for a,b in G2.edges():
                f.write("Req:%s %s xx %s\n"%(u.id, a, b))
            

URS3652.7 {'URS3652'}


In [146]:
add_implicit_local_refs(allitems)
G = items_to_graph(allitems)
reqs = dictify(allitems)

def condensed_graph(impacted, reqs):

    # Impacted is a set of all nodes impacted, but has no edge data
    # Edge data is contained in G

    G2 = nx.DiGraph()

    # "fold" references into horizontal hierarchy whereever 
    # a parent of the same requirement type has the same URS as 
    # its children

    for node in impacted:
        req = reqs.get(node)
        if req is None:
            print("Warning! Skipping non-existent node %s"%node)
            continue
        hiparent = req.tag.rsplit(".", 1)[0]
        if hiparent == req.tag or hiparent not in impacted:
            # no parents to this req. just add refs
            for ref in req.refs:
                if ref not in impacted:
                    continue
                G2.add_edge(ref, req.tag)
        else:
            # maybe has refs to fold
            parent = reqs[hiparent]
            for ref in req.refs:
                if ref not in impacted:
                    continue
                if ref in parent.refs:
                    G2.add_edge(parent.tag, req.tag)
                else:
                    G2.add_edge(ref, req.tag)
    return G2

def condensed_impact_graph(G, epicenter, reqs):
    impacted = impacted_nodes(G, epicenter)
    # use a different color for epicenter
    G.add_node(epicenter, color="yellow")
    return condensed_graph(impacted, reqs)
    
# G2 = condensed_impact_graph(G, "FRS4401.2.3")
# nx.write_graphml(G2, "test.xml")

In [143]:
def condensed_implement_graph(G, node, reqs):
    d = nx.shortest_path_length(G, source=node)
    return condensed_graph(d, reqs)

In [144]:
def combined_implement_graph(G, nodes, reqs):
    d = set()
    for n in nodes:
        try:
            nrefs = impacted_nodes(G, n)
        except nx.NodeNotFound:
            pass
        else:
            d.update(nrefs)
    return condensed_graph(d, reqs)

In [45]:
target = "URS4370.6.2"
G2 = condensed_implement_graph(G, target, reqs)
nx.write_graphml(G2, "test.xml")
with open("colors.csv", 'w') as f:
    for node in G2.nodes():
        if node == target:
            color = "yellow"
        else:
            color="blue"
        f.write("%s,%s\n"%(node, color))

In [334]:
def copysubgraph(G, nodes):
    G2 = nx.DiGraph()
    G2.add_nodes_from(nodes)
    for s,t in G.edges():
        if s in G2 and t in G2:
            G2.add_edge(s,t)
    return G2

def combined_implement_graph3(G, new):
    d = set()
    for n in new:
        try:
            nrefs = impacted_nodes(G, n)
        except nx.NodeNotFound:
            pass
        else:
            d.update(nrefs)
    return combined_implement_graph2(G, d)

def combined_implement_graph2(G, new):
    #G2 = copysubgraph(G, new)
    return condensed_graph2(G2)
    
def condensed_graph2(G):
    return nx.algorithms.dag.transitive_reduction(G)
    
    

In [200]:
import py2cytoscape
from py2cytoscape.data.cyrest_client import CyRestClient

cy = CyRestClient()

In [377]:
new = {n.tag for n in newitems}

for a in allitems:
    if a.tag in new:
        a.obs = True

apn = allitems + newitems
add_implicit_local_refs(apn)
reqs = dictify(apn)

apn = [n for n in apn if n.type != "TEST"]

G = items_to_graph(apn, reqs)

print("running big long operation...")
#G2 = combined_implement_graph2(G, new, reqs)#
#G2 = copysubgraph(G, new)

G2 = combined_implement_graph3(G, new)
G2 = condensed_graph2(G)

print("big long operation done")
nx.write_graphml(G2, "test.xml")

cy.session.delete()

data = py2cytoscape.util.from_networkx(G2)
for node in data['elements']['nodes']:
    node = node['data']
    req = reqs[node['id']]
    node['type'] = req.type
    node['group'] = req.num.split('.',1)[0]
    node['new'] = str(node['id'] in new)
    
nw = cy.network.create(None, name="test", collection="Test1", data=data)
cy.layout.apply("force-directed", nw)

running big long operation...
big long operation done


In [366]:
print(requests.get("http://localhost:1234/v1/commands/network").content.decode())

<p style="color:blue;margin-top:0px;margin-bottom:5px;">Available commands for 'network':</p>
<p style="color:black;margin-left:10px;margin-top:0px;margin-bottom:5px;">  add</p>
<p style="color:black;margin-left:10px;margin-top:0px;margin-bottom:5px;">  add edge</p>
<p style="color:black;margin-left:10px;margin-top:0px;margin-bottom:5px;">  add node</p>
<p style="color:black;margin-left:10px;margin-top:0px;margin-bottom:5px;">  clone</p>
<p style="color:black;margin-left:10px;margin-top:0px;margin-bottom:5px;">  connect nodes</p>
<p style="color:black;margin-left:10px;margin-top:0px;margin-bottom:5px;">  create</p>
<p style="color:black;margin-left:10px;margin-top:0px;margin-bottom:5px;">  create attribute</p>
<p style="color:black;margin-left:10px;margin-top:0px;margin-bottom:5px;">  create empty</p>
<p style="color:black;margin-left:10px;margin-top:0px;margin-bottom:5px;">  delete</p>
<p style="color:black;margin-left:10px;margin-top:0px;margin-bottom:5px;">  deselect</p>
<p style="c

In [370]:
view = nw.get_view(nw.get_views()[0], format='view')

In [358]:
import requests
rsp = requests.get(cy.layout._LayoutClient__url+"/"+'force-directed'+'/'+str(nw.get_id()) + '?'+"column=feature")
rsp.status_code

200

In [374]:
import py2cytoscape.cytoscapejs as cyjs

In [376]:
requests.get("http://localhost:1234/v1/colle")

{'Breadthfirst': 'breadthfirst',
 'Circle': 'circle',
 'Concentric': 'concentric',
 'Grid': 'grid',
 'Preset': 'preset',
 'Spring': 'cose'}

In [331]:
impacted_nodes(G, "FRS4400.1.2.2")

{'FRS4400',
 'FRS4400.1',
 'FRS4400.1.2',
 'FRS4400.1.2.2',
 'URS3664',
 'URS3664.3',
 'URS4400',
 'URS4400.1'}